In [35]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [82]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import gc
import xgboost as xgb
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression

In [37]:
items           = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train_v2.csv')
test            = pd.read_csv('test.csv')
sample_submission            = pd.read_csv('sample_submission.csv')

In [38]:
transactions = sales_train.copy()
transactions[['day','month', 'year']] = transactions['date'].str.split('.', expand=True).astype(int)


In [39]:
w_cat_ids = transactions.set_index('item_id').join(items.set_index('item_id'))
w_cat_ids = w_cat_ids.reset_index()
w_cat_ids['row_id'] = w_cat_ids.index
w_cat_ids['y'] = w_cat_ids.groupby(['date_block_num', 'item_id', 'shop_id'])['item_cnt_day'].transform('sum').clip(0,20)
w_cat_ids = w_cat_ids.drop_duplicates(['date_block_num', 'item_id', 'shop_id'])
w_cat_ids = w_cat_ids[w_cat_ids['year'] != 2013]


In [ ]:
w_cat_ids['item_mean'] = w_cat_ids.groupby(['item_id'])['y'].transform('mean')
w_cat_ids['item_mean_month'] = w_cat_ids.groupby(['item_id', 'month'])['y'].transform('mean')
w_cat_ids['shop_mean_month'] = w_cat_ids.groupby(['shop_id', 'month'])['y'].transform('mean')
w_cat_ids['category_mean_month'] = w_cat_ids.groupby(['item_category_id', 'month'])['y'].transform('mean')
w_cat_ids['shop_category_mean_month'] = w_cat_ids.groupby(['shop_id','item_category_id', 'month'])['y'].transform('mean')
w_cat_ids['shop_item_mean_month'] = w_cat_ids.groupby(['shop_id','item_id', 'month'])['y'].transform('mean')

In [41]:
X = w_cat_ids[(w_cat_ids['year'] == 2015) | ((w_cat_ids['year'] == 2014) & (w_cat_ids['month'].isin([11,12])))]

In [42]:
rollings = X[['item_mean_month', 'date_block_num', 'item_id']].copy()
rollings = rollings.drop_duplicates(['item_id', 'date_block_num'])
rollings = rollings.set_index(['date_block_num', 'item_id'])
a = rollings.sort_values(['item_id', 'date_block_num'])\
        .groupby(['item_id'])['item_mean_month'].shift(1)
aa = rollings.sort_values(['item_id', 'date_block_num'])\
        .groupby(['item_id'])['item_mean_month'].shift(3)
aaa = rollings.sort_values(['item_id', 'date_block_num'])\
        .groupby(['item_id'])['item_mean_month'].shift(6)

In [43]:
b = pd.DataFrame(a)
b.rename(columns={'item_mean_month': 'item_mean_month_minus_1'}, inplace=True)

X = pd.merge(X, b, on=['item_id', 'date_block_num'], how='left')

In [44]:
b = pd.DataFrame(aa)
b.rename(columns={'item_mean_month': 'item_mean_month_minus_3'}, inplace=True)

X = pd.merge(X, b, on=['item_id', 'date_block_num'], how='left')

In [45]:
b = pd.DataFrame(aaa)
b.rename(columns={'item_mean_month': 'item_mean_month_minus_6'}, inplace=True)

X = pd.merge(X, b, on=['item_id', 'date_block_num'], how='left')

In [46]:
rollings = X[['shop_mean_month', 'date_block_num', 'shop_id']].copy()
rollings = rollings.drop_duplicates(['shop_id', 'date_block_num'])
rollings = rollings.set_index(['date_block_num', 'shop_id'])
a = rollings.sort_values(['shop_id', 'date_block_num'])\
        .groupby(['shop_id'])['shop_mean_month'].shift(1)
aa = rollings.sort_values(['shop_id', 'date_block_num'])\
        .groupby(['shop_id'])['shop_mean_month'].shift(3)
aaa = rollings.sort_values(['shop_id', 'date_block_num'])\
        .groupby(['shop_id'])['shop_mean_month'].shift(6)

In [47]:
b = pd.DataFrame(a)
b.rename(columns={'shop_mean_month': 'shop_mean_month_minus_1'}, inplace=True)

X = pd.merge(X, b, on=['shop_id', 'date_block_num'], how='left')

In [48]:
b = pd.DataFrame(aa)
b.rename(columns={'shop_mean_month': 'shop_mean_month_minus_3'}, inplace=True)

X = pd.merge(X, b, on=['shop_id', 'date_block_num'], how='left')

In [49]:
b = pd.DataFrame(aaa)
b.rename(columns={'shop_mean_month': 'shop_mean_month_minus_6'}, inplace=True)

X = pd.merge(X, b, on=['shop_id', 'date_block_num'], how='left')

In [50]:
rollings = X[['category_mean_month', 'date_block_num', 'item_category_id']].copy()
rollings = rollings.drop_duplicates(['item_category_id', 'date_block_num'])
rollings = rollings.set_index(['date_block_num', 'item_category_id'])
a = rollings.sort_values(['item_category_id', 'date_block_num'])\
        .groupby(['item_category_id'])['category_mean_month'].shift(1)
aa = rollings.sort_values(['item_category_id', 'date_block_num'])\
        .groupby(['item_category_id'])['category_mean_month'].shift(3)
aaa = rollings.sort_values(['item_category_id', 'date_block_num'])\
        .groupby(['item_category_id'])['category_mean_month'].shift(6)

In [51]:
b = pd.DataFrame(a)
b.rename(columns={'category_mean_month': 'category_mean_month_minus_1'}, inplace=True)

X = pd.merge(X, b, on=['item_category_id', 'date_block_num'], how='left')

In [52]:
b = pd.DataFrame(aa)
b.rename(columns={'category_mean_month': 'category_mean_month_minus_3'}, inplace=True)

X = pd.merge(X, b, on=['item_category_id', 'date_block_num'], how='left')

In [53]:
b = pd.DataFrame(aaa)
b.rename(columns={'category_mean_month': 'category_mean_month_minus_6'}, inplace=True)

X = pd.merge(X, b, on=['item_category_id', 'date_block_num'], how='left')

In [54]:
rollings = X[['shop_category_mean_month', 'date_block_num', 'shop_id', 'item_category_id']].copy()
rollings = rollings.drop_duplicates(['item_category_id', 'shop_id', 'date_block_num'])
rollings = rollings.set_index(['date_block_num', 'item_category_id', 'shop_id'])
a = rollings.sort_values(['item_category_id', 'shop_id', 'date_block_num'])\
        .groupby(['item_category_id', 'shop_id'])['shop_category_mean_month'].shift(1)
aa = rollings.sort_values(['item_category_id', 'shop_id', 'date_block_num'])\
        .groupby(['item_category_id', 'shop_id'])['shop_category_mean_month'].shift(3)
aaa = rollings.sort_values(['item_category_id', 'shop_id', 'date_block_num'])\
        .groupby(['item_category_id', 'shop_id'])['shop_category_mean_month'].shift(6)

In [55]:
b = pd.DataFrame(a)
b.rename(columns={'shop_category_mean_month': 'shop_category_mean_month_minus_1'}, inplace=True)

X = pd.merge(X, b, on=['item_category_id', 'shop_id', 'date_block_num'], how='left')

In [56]:
b = pd.DataFrame(aa)
b.rename(columns={'shop_category_mean_month': 'shop_category_mean_month_minus_3'}, inplace=True)

X = pd.merge(X, b, on=['item_category_id', 'shop_id', 'date_block_num'], how='left')

In [57]:
b = pd.DataFrame(aaa)
b.rename(columns={'shop_category_mean_month': 'shop_category_mean_month_minus_6'}, inplace=True)

X = pd.merge(X, b, on=['item_category_id', 'shop_id', 'date_block_num'], how='left')

In [58]:
X[X['item_id'] == 30].drop_duplicates(['item_id', 'date_block_num'])[['item_id', 'month', 'year', 'item_mean_month', 'item_mean_month_minus_1', 'item_mean_month_minus_3', 'item_mean_month_minus_6']]

,item_id,month,year,item_mean_month,item_mean_month_minus_1,item_mean_month_minus_3,item_mean_month_minus_6
10,30,11,2014,1.300000,NaN,NaN,NaN
20,30,12,2014,1.600000,1.300000,NaN,NaN
30,30,1,2015,1.894737,1.600000,NaN,NaN
41,30,2,2015,1.266667,1.894737,1.300000,NaN
53,30,3,2015,1.714286,1.266667,1.600000,NaN
58,30,4,2015,1.388889,1.714286,1.894737,NaN
62,30,5,2015,1.235294,1.388889,1.266667,1.300000
66,30,6,2015,1.133333,1.235294,1.714286,1.600000
70,30,7,2015,1.214286,1.133333,1.388889,1.894737
73,30,8,2015,1.285714,1.214286,1.235294,1.266667


In [59]:
X[X['shop_id'] == 30].drop_duplicates(['shop_id', 'date_block_num'])[['shop_id', 'month', 'year', 'shop_mean_month', 'shop_mean_month_minus_1', 'shop_mean_month_minus_3', 'shop_mean_month_minus_6']]

,shop_id,month,year,shop_mean_month,shop_mean_month_minus_1,shop_mean_month_minus_3,shop_mean_month_minus_6
11,30,11,2014,1.873958,NaN,NaN,NaN
35,30,1,2015,1.736099,2.118103,NaN,NaN
130,30,2,2015,1.670232,1.736099,1.873958,NaN
245,30,12,2014,2.118103,1.873958,NaN,NaN


In [60]:
X[X['item_category_id'] == 30].drop_duplicates(['item_category_id', 'date_block_num'])[['item_category_id', 'month', 'year', 'category_mean_month', 'category_mean_month_minus_1']]

,item_category_id,month,year,category_mean_month,category_mean_month_minus_1
3762,30,10,2015,3.070205,2.746511
11027,30,1,2015,3.009862,3.101948
11028,30,7,2015,2.670056,2.788012
12837,30,11,2014,2.843284,NaN
12854,30,12,2014,3.101948,2.843284
12897,30,2,2015,2.905608,3.009862
12904,30,3,2015,2.889822,2.905608
12913,30,4,2015,2.892322,2.889822
12925,30,5,2015,2.702312,2.892322
12933,30,6,2015,2.788012,2.702312


In [61]:
X[(X['item_category_id'] == 30) & (X['shop_id'] == 30)].drop_duplicates(['item_category_id', 'shop_id', 'date_block_num'])[['item_category_id', 'shop_id' ,  'month', 'year', 'shop_category_mean_month', 'shop_category_mean_month_minus_1']]

,item_category_id,shop_id,month,year,shop_category_mean_month,shop_category_mean_month_minus_1
16939,30,30,11,2014,1.864198,NaN
16992,30,30,12,2014,2.421687,1.864198
17043,30,30,1,2015,2.597633,2.421687
17087,30,30,2,2015,2.488550,2.597633


In [62]:
X.columns

Index(['item_id', 'date', 'date_block_num', 'shop_id', 'item_price',
       'item_cnt_day', 'day', 'month', 'year', 'item_name', 'item_category_id',
       'row_id', 'y', 'item_mean', 'item_mean_month', 'shop_mean_month',
       'category_mean_month', 'shop_category_mean_month',
       'item_mean_month_minus_1', 'item_mean_month_minus_3',
       'item_mean_month_minus_6', 'shop_mean_month_minus_1',
       'shop_mean_month_minus_3', 'shop_mean_month_minus_6',
       'category_mean_month_minus_1', 'category_mean_month_minus_3',
       'category_mean_month_minus_6', 'shop_category_mean_month_minus_1',
       'shop_category_mean_month_minus_3', 'shop_category_mean_month_minus_6'],
      dtype='object')

In [63]:
columns = X.columns
percent_missing = X.isna().sum() * 100 / len(X)
pd.DataFrame({'column_name': columns,
                                 'percent_missing': percent_missing})

,column_name,percent_missing
item_id,item_id,0.000000
date,date,0.000000
date_block_num,date_block_num,0.000000
shop_id,shop_id,0.000000
item_price,item_price,0.000000
item_cnt_day,item_cnt_day,0.000000
day,day,0.000000
month,month,0.000000
year,year,0.000000
item_name,item_name,0.000000


In [64]:
test_w_cat_ids = test.set_index('item_id').join(items.set_index('item_id'))
test_w_cat_ids['item_id'] = test_w_cat_ids.index

In [65]:
#features = ['item_id', 'date_block_num', 'shop_id', #'item_price',
lgbm_features = ['item_id', 'shop_id',  'month', 'item_category_id',
       'item_mean', 'item_mean_month', 'shop_mean_month',
       'category_mean_month', 'shop_category_mean_month',
       'item_mean_month_minus_1', 'item_mean_month_minus_3',
       'item_mean_month_minus_6', 'shop_mean_month_minus_1',
       'shop_mean_month_minus_3', 'shop_mean_month_minus_6',
       'category_mean_month_minus_1', 'category_mean_month_minus_3',
       'category_mean_month_minus_6', 'shop_category_mean_month_minus_1',
       'shop_category_mean_month_minus_3', 'shop_category_mean_month_minus_6']
    
lr_features = [
       'item_mean', 'item_mean_month', 'shop_mean_month',
       'category_mean_month', 'shop_category_mean_month',
       'item_mean_month_minus_1', 
       'shop_mean_month_minus_1',
       'category_mean_month_minus_1',
       'shop_category_mean_month_minus_1',]


In [66]:
val_indices = X[(X['year'] == 2015) & (X['month'].isin([9,10]))].index

In [67]:
x = X
y = X['y']


x_train = x[~x.index.isin(val_indices)]
y_train = y[~y.index.isin(val_indices)]

x_val = x[x.index.isin(val_indices)]
y_val = y[y.index.isin(val_indices)]

In [100]:
lgtrain = lgbm.Dataset(x_train[lgbm_features], label=y_train, categorical_feature=[0,1,2,3])
lgval = lgbm.Dataset(x_val[lgbm_features], label=y_val, categorical_feature=[0,1,2,3])



#[0.00542047893814942, 29, 24, 0.39949465609514856, 1, 0.67943500, 10]
params = {
        "num_threads": 8,
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        #"max_bin": 10,
        #"num_leaves": 10,
        "bagging_fraction": 0.3,
        "bagging_freq": 1,
        "min_data_in_leaf": 5000,
        "feature_fraction": 0.5,
        #"lambda_l2": 3,
        #"max_depth": 2,
        "learning_rate" : 0.01
}

evals_result = {}
#model_lgb = lgbm.train(params, lgtrain, 50000, 
 #                     valid_sets=[lgval], 
  #                    early_stopping_rounds=100, 
   #                   verbose_eval=500, 
    #                  evals_result=evals_result)



/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.81865
[1000]	valid_0's rmse: 1.73614
[1500]	valid_0's rmse: 1.69838
[2000]	valid_0's rmse: 1.67381
[2500]	valid_0's rmse: 1.65572
[3000]	valid_0's rmse: 1.64298
[3500]	valid_0's rmse: 1.63354
[4000]	valid_0's rmse: 1.62574
[4500]	valid_0's rmse: 1.61952
[5000]	valid_0's rmse: 1.61477
[5500]	valid_0's rmse: 1.61013
[6000]	valid_0's rmse: 1.60703
[6500]	valid_0's rmse: 1.6036
[7000]	valid_0's rmse: 1.60071
[7500]	valid_0's rmse: 1.5984
[8000]	valid_0's rmse: 1.59613
[8500]	valid_0's rmse: 1.594
[9000]	valid_0's rmse: 1.59159
[9500]	valid_0's rmse: 1.58983
[10000]	valid_0's rmse: 1.58833
[10500]	valid_0's rmse: 1.58729
[11000]	valid_0's rmse: 1.58579
[11500]	valid_0's rmse: 1.58452
[12000]	valid_0's rmse: 1.58315
[12500]	valid_0's rmse: 1.58218
[13000]	valid_0's rmse: 1.58107
Early stopping, best iteration is:
[13255]	valid_0's rmse: 1.5806


In [104]:
#pickle.dump(model_lgb, open( "model_lgb", "wb"))

model_lgb = pickle.load( open( "model_lgb", "rb" ) )

In [99]:
x_train_lr = x[~x.index.isin(val_indices)].copy()[lr_features]
x_train_lr.fillna(x_train_lr.mean(), inplace=True)

x_val_lr = x[x.index.isin(val_indices)].copy()[lr_features]
x_val_lr.fillna(x_train_lr.mean(), inplace=True)


lr_model = Ridge(alpha=0.5)
lr_model.fit(x_train_lr, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

lr_val_preds = lr_model.predict(x_val_lr)
lr_val_preds.clip(0,20,out=lr_val_preds)
rms = sqrt(mean_squared_error(y_val, lr_val_preds))
print("rmse: ", rms)

rmse:  1.6796097341448792


In [98]:
train_meta['date_block_num'].unique()

array([28, 29, 30, 31, 32, 33])

In [72]:
#stacking
blocks = [28, 29, 30, 31, 32, 33]

train_meta = X.copy()
train_meta = train_meta.sort_values(by=['date_block_num']).reset_index()

In [ ]:
lr_preds = np.zeros(len(train_meta))


for block in blocks:
    
    fold = train_meta[train_meta['date_block_num'] == block]
    train = train_meta[train_meta['date_block_num'] < block]
    y = train['y']
    
    #lr
    x_lr = train[lr_features].copy()
    x_lr.fillna(x_lr.mean(), inplace=True)
    
    lr_model = Ridge(alpha=0.5)
    lr_model.fit(x_lr, y)
    oof = fold[lr_features].copy()
    oof.fillna(oof.mean(), inplace=True)
    oof_pred = lr_model.predict(oof)
    oof_pred.clip(0,20,out=oof_pred)

    lr_preds[fold.index] = oof_pred
    
train_meta['lr_pred'] = lr_preds

In [73]:
lgbm_preds = np.zeros(len(train_meta))

if False:
#for block in blocks:
    
    fold = train_meta[train_meta['date_block_num'] == block]
    train = train_meta[train_meta['date_block_num'] < block]
    y = train['y']
    
    #lgbm
    val_len = int(10 * len(train) / 100)
    print(val_len)
    val_indices = sorted(train.index)[-val_len:]


    x_train = train[~train.index.isin(val_indices)]
    y_train = y[~y.index.isin(val_indices)]

    x_val = train[train.index.isin(val_indices)]
    y_val = y[y.index.isin(val_indices)]
    
    
    
    lgtrain = lgbm.Dataset(x_train[lgbm_features], label=y_train, categorical_feature=[0,1,2,3])
    lgval = lgbm.Dataset(x_val[lgbm_features], label=y_val, categorical_feature=[0,1,2,3])
    
    
    
    params = {
        "num_threads": 8,
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        #"max_bin": 10,
        #"num_leaves": 10,
        "bagging_fraction": 0.3,
        "bagging_freq": 1,
        "min_data_in_leaf": 5000,
        "feature_fraction": 0.5,
        #"lambda_l2": 3,
        #"max_depth": 2,
        "learning_rate" : 0.01
    }

    evals_result = {}
    model_lgb = lgbm.train(params, lgtrain, 50000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=500, 
                      evals_result=evals_result)
    
    oof = fold[lgbm_features]

    oof_preds = model_lgb.predict(oof)
    oof_preds.clip(0,20,out=oof_preds)

    lgbm_preds[fold.index] = oof_preds
    
train_meta['lgbm_pred'] = lgbm_preds

In [77]:
import pickle

lgbm_preds = pickle.load( open( "lgbm_pred", "rb" ) )
train_meta['lgbm_pred'] = lgbm_preds

In [97]:
train_meta_train = train_meta[train_meta['date_block_num'].isin([28, 29, 30, 31, 32])]
train_meta_val = train_meta[train_meta['date_block_num'].isin([33])]

meta_features = ['lr_pred', 'lgbm_pred']


meta_model = LinearRegression().fit(train_meta_train[meta_features], train_meta_train['y'])

meta_preds = meta_model.predict(train_meta_val[meta_features])
meta_preds.clip(0,20,out=meta_preds)
rms = sqrt(mean_squared_error(train_meta_val['y'], meta_preds))
print("rmse: ", rms)

rmse:  1.5580748985261248


In [105]:
test_w_cat_ids = test.set_index('item_id').join(items.set_index('item_id'))
test_w_cat_ids['item_id'] = test_w_cat_ids.index

In [106]:
test_w_cat_ids['month'] = 11

In [107]:
item_level = X.drop_duplicates(['item_id'])[['item_id','item_mean', 'item_price']]

test_w_cat_ids = pd.merge(test_w_cat_ids, item_level, on=['item_id'], how='left')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3265: FutureWarning: 'item_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [108]:
item_month_level = X[(X.month == 11)].drop_duplicates(['item_id'])[['item_id','item_mean_month']]

test_w_cat_ids = pd.merge(test_w_cat_ids, item_month_level, on=['item_id'], how='left')

In [109]:
item_month_level_ = X[X.month == 10].drop_duplicates(['item_id'])[['item_id','item_mean_month']]\
                                   .copy().rename(columns={'item_mean_month':'item_mean_month_minus_1'})


test_w_cat_ids = pd.merge(test_w_cat_ids, item_month_level_, on=['item_id'], how='left')

In [110]:
item_month_level_ = X[X.month == 8].drop_duplicates(['item_id'])[['item_id','item_mean_month']]\
                                   .copy().rename(columns={'item_mean_month':'item_mean_month_minus_3'})


test_w_cat_ids = pd.merge(test_w_cat_ids, item_month_level_, on=['item_id'], how='left')

In [111]:
item_month_level_ = X[X.month == 5].drop_duplicates(['item_id'])[['item_id','item_mean_month']]\
                                   .copy().rename(columns={'item_mean_month':'item_mean_month_minus_6'})


test_w_cat_ids = pd.merge(test_w_cat_ids, item_month_level_, on=['item_id'], how='left')

In [112]:
shop_month_level_ = X[(X.month == 11)].drop_duplicates(['shop_id'])[['shop_id','shop_mean_month']]

test_w_cat_ids = pd.merge(test_w_cat_ids, shop_month_level_, on=['shop_id'], how='left')

In [113]:
shop_month_level_ = X[X.month == 10].drop_duplicates(['shop_id'])[['shop_id','shop_mean_month']]\
                                   .copy().rename(columns={'shop_mean_month':'shop_mean_month_minus_1'})


test_w_cat_ids = pd.merge(test_w_cat_ids, shop_month_level_, on=['shop_id'], how='left')

In [114]:
shop_month_level_ = X[X.month == 8].drop_duplicates(['shop_id'])[['shop_id','shop_mean_month']]\
                                   .copy().rename(columns={'shop_mean_month':'shop_mean_month_minus_3'})


test_w_cat_ids = pd.merge(test_w_cat_ids, shop_month_level_, on=['shop_id'], how='left')

In [115]:
shop_month_level_ = X[X.month == 5].drop_duplicates(['shop_id'])[['shop_id','shop_mean_month']]\
                                   .copy().rename(columns={'shop_mean_month':'shop_mean_month_minus_6'})


test_w_cat_ids = pd.merge(test_w_cat_ids, shop_month_level_, on=['shop_id'], how='left')

In [116]:
cat_month_level_ = X[(X.month == 11)].drop_duplicates(['item_category_id'])[['item_category_id','category_mean_month']]

test_w_cat_ids = pd.merge(test_w_cat_ids, cat_month_level_, on=['item_category_id'], how='left')

In [117]:
cat_month_level_ = X[X.month == 10].drop_duplicates(['item_category_id'])[['item_category_id','category_mean_month']]\
                                   .copy().rename(columns={'category_mean_month':'category_mean_month_minus_1'})


test_w_cat_ids = pd.merge(test_w_cat_ids, cat_month_level_, on=['item_category_id'], how='left')

In [118]:
cat_month_level_ = X[X.month == 8].drop_duplicates(['item_category_id'])[['item_category_id','category_mean_month']]\
                                   .copy().rename(columns={'category_mean_month':'category_mean_month_minus_3'})


test_w_cat_ids = pd.merge(test_w_cat_ids, cat_month_level_, on=['item_category_id'], how='left')

In [119]:
cat_month_level_ = X[X.month == 5].drop_duplicates(['item_category_id'])[['item_category_id','category_mean_month']]\
                                   .copy().rename(columns={'category_mean_month':'category_mean_month_minus_6'})


test_w_cat_ids = pd.merge(test_w_cat_ids, cat_month_level_, on=['item_category_id'], how='left')

In [120]:
shop_cat_month_level_ = X[(X.month == 11)].drop_duplicates(['item_category_id', 'shop_id'])[['item_category_id', 'shop_id', 'shop_category_mean_month']]

test_w_cat_ids = pd.merge(test_w_cat_ids, shop_cat_month_level_, on=['item_category_id', 'shop_id'], how='left')

In [121]:
shop_cat_month_level_ = X[X.month == 10].drop_duplicates(['item_category_id', 'shop_id'])[['item_category_id', 'shop_id', 'shop_category_mean_month']]\
                                   .copy().rename(columns={'shop_category_mean_month':'shop_category_mean_month_minus_1'})


test_w_cat_ids = pd.merge(test_w_cat_ids, shop_cat_month_level_, on=['item_category_id', 'shop_id'], how='left')

In [122]:
shop_cat_month_level_ = X[X.month == 8].drop_duplicates(['item_category_id', 'shop_id'])[['item_category_id', 'shop_id', 'shop_category_mean_month']]\
                                   .copy().rename(columns={'shop_category_mean_month':'shop_category_mean_month_minus_3'})


test_w_cat_ids = pd.merge(test_w_cat_ids, shop_cat_month_level_, on=['item_category_id', 'shop_id'], how='left')

In [123]:
shop_cat_month_level_ = X[X.month == 5].drop_duplicates(['item_category_id', 'shop_id'])[['item_category_id', 'shop_id', 'shop_category_mean_month']]\
                                   .copy().rename(columns={'shop_category_mean_month':'shop_category_mean_month_minus_6'})


test_w_cat_ids = pd.merge(test_w_cat_ids, shop_cat_month_level_, on=['item_category_id', 'shop_id'], how='left')

In [130]:
# stacking, we have model_lgb and lr_model fit on entire training, we predict for test

x_lr_test = test_w_cat_ids[lr_features].copy()
x_lr_test.fillna(x_lr.mean(), inplace=True)

test_lr_pred = lr_model.predict(x_lr_test[lr_features])
test_lr_pred.clip(0,20,out=test_lr_pred)


test_lgbm_preds = model_lgb.predict(test_w_cat_ids[lgbm_features])
test_lgbm_preds.clip(0,20,out=test_lgbm_preds)



array([ 1.06749073,  1.0895982 ,  1.08557396, ...,  1.27861283,
        1.26062962,  1.35388615])

In [131]:
test_meta = pd.DataFrame()
test_meta['lr_pred'] = test_lr_pred
test_meta['lgbm_pred'] = test_lgbm_preds


final_preds = meta_model.predict(test_meta)


submission = test.loc[:,['ID']]
submission['item_cnt_month'] = final_preds.astype(int)

submission.to_csv('submission.csv', index=False)

In [ ]:
test_w_cat_ids[features].head()

In [125]:
columns = X.columns
percent_missing = X.isna().sum() * 100 / len(X)
pd.DataFrame({'column_name': columns,
                                 'percent_missing': percent_missing})

,column_name,percent_missing
item_id,item_id,0.000000
date,date,0.000000
date_block_num,date_block_num,0.000000
shop_id,shop_id,0.000000
item_price,item_price,0.000000
item_cnt_day,item_cnt_day,0.000000
day,day,0.000000
month,month,0.000000
year,year,0.000000
item_name,item_name,0.000000


In [126]:
columns = test_w_cat_ids.columns
percent_missing = test_w_cat_ids.isna().sum() * 100 / len(test_w_cat_ids)
pd.DataFrame({'column_name': columns,
                                 'percent_missing': percent_missing})

,column_name,percent_missing
ID,ID,0.000000
shop_id,shop_id,0.000000
item_name,item_name,0.000000
item_category_id,item_category_id,0.000000
item_id,item_id,0.000000
month,month,0.000000
item_mean,item_mean,8.843137
item_price,item_price,8.843137
item_mean_month,item_mean_month,51.509804
item_mean_month_minus_1,item_mean_month_minus_1,21.745098


In [ ]:
lr_features = [
       'item_mean', 'item_mean_month', 'shop_mean_month',
       'category_mean_month', 'shop_category_mean_month',
       'item_mean_month_minus_1', 
       'shop_mean_month_minus_1',
       'category_mean_month_minus_1',
       'shop_category_mean_month_minus_1',]

In [ ]:
lr_preds = lr_model.predict(test_w_cat_ids[lr_features])

In [133]:
preds = model_lgb.predict(test_w_cat_ids[lgbm_features])
preds.clip(0,20,out=preds)

array([ 1.06749073,  1.0895982 ,  1.08557396, ...,  1.27861283,
        1.26062962,  1.35388615])

In [134]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds.astype(int)

submission.to_csv('submission.csv', index=False)

In [ ]:
print('Plotting feature importances...')
ax = lgbm.plot_importance(model_lgb, max_num_features=20)
plt.show()

3.45 val > 3.70 submit
3.06 val > 3.65 submit